In [19]:
#!pip install playwright
#!pip install geonamescache
import tweepy
import numpy as np
import pandas as pd
import json
from bs4 import BeautifulSoup
from helper import *
from IPython.display import clear_output #clears output

DRIVER = webdriver.Chrome()#the chrome

API_Key ='irXxqVrXin1KsDRM9g6HdF8p8'
API_Key_Secret = 'sLQvb8voyrTDz5S93vlt5jE5RlMi8xBuOQtSq2pgERe2QVawIC'

Bearer_Token = 'AAAAAAAAAAAAAAAAAAAAACsVlAEAAAAADib49u9Asa6Hbts8UwBEdtc0dsk%3D1d8lVaySeRg1CpJQ6HDLSwK6Mj9wJiptj49PAdwMuvW95zJoar'

Access_Token = '1122492746210660352-COptyz3RtF1AVokHw3027iOdbsZZl1'
Access_Token_Secret = 'slfBz8pcgCrUmewZgSgVKsmIFhfCcFuinAx16q5eh1Vvr'

In [20]:
# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(API_Key, API_Key_Secret)
# set access to user's access key and access secret
auth.set_access_token(Access_Token, Access_Token_Secret)
# calling the api
api = tweepy.API(auth)

Client = tweepy.Client(bearer_token=Bearer_Token) 

In [21]:
#search_tweets(q, *, geocode, lang, locale, result_type, count, until, since_id, max_id, include_entities)



#user data
user_name = [];
location = [];
age = [];
gender = [];
followers = [];
following = [];
verified = []
joined = [] #when they joined
tweet_count_user = []

#tweet data
tweet = [];
word_count = [];
lang = []
likes = []
retweet_count = []
is_quote_status = []

trending_for_area = []

users = [];
tweets_list = tweepy.Cursor(api.search_30_day('30day','#zelda'), count=100).items(100)

for x in tweets_list:
    
    tweet.append(x.text);
    users.append(x.user);
    word_count.append(countWordsInString(x.text))
    
    lang.append(x.lang)
    likes.append(x.favorite_count) 
    retweet_count.append(x.retweet_count)
    is_quote_status.append(x.is_quote_status)
    
i = 0
for x in users:
    j = x._json
    #opens the page and waits for it to load
    screen_name = j['screen_name']
    country = check_country_real(j['location'])
    
    
    DRIVER.get('https://twitter.com/' + screen_name) 
    
    #get the easy stuff
    user_name.append(screen_name)
    following.append(j['friends_count']);
    followers.append(j['followers_count']);
    location.append(country);
    joined.append(x.created_at.isoformat()[:10])
    tweet_count_user.append(x.statuses_count)
    trending_for_area.append(getTrendingTopics(api,country))
    verified = x.verified
    
    
    
    tmp = getAge(DRIVER)#gets the age
    if(tmp == 2023 - x.created_at.year): #checks if the age is the same as the accounts age
        age.append(np.nan)
    else:
        age.append(tmp)
    
    
    gender.append(getGender(j['description'],j['location'],j['name']));
    
    ##this part simply displays at the bottom how many profiles were checked
    clear_output(wait=True) #clears the output
    
    i+=1
    print(i)

print('done')

TweepyException: This method does not perform pagination

In [4]:
dick = {"name":user_name,
        "age":age,
        'location':location,
        "gender":gender,
        'join_date':joined,
        'total tweets': tweet_count_user,
        "followers":followers,
        "following":following,
        'verified' : verified,
        'tweet':tweet,
        'likes':likes,
        'retweets':retweet_count,
        'word count': word_count,
        'launguage': lang,
        'is quote':is_quote_status,
        'trending': trending_for_area
       }

df = pd.DataFrame(dick)

In [5]:
df

,name,age,location,gender,join_date,total tweets,followers,following,verified,tweet,likes,retweets,word count,launguage,is quote,trending
0,Vusumzi__,NaN,None,None,2014-07-10,69377,1585,953,False,RT @M1VUYO: Sizwe Dhlomo and Andrew Tate walk ...,0,727,19,en,True,None
1,BigJ74835035,NaN,None,None,2022-06-26,799,16,25,False,@chrisprimod Nobody will take your protest ser...,0,0,20,en,False,None
2,johnhawkinsrwn,NaN,None,None,2008-07-18,6349,25653,963,False,"@nypost You make it sound so scurrilous, sir, ...",0,0,23,en,False,None
3,casualcaila,NaN,None,None,2022-03-07,3916,53,227,False,andrew tate reminds me of henchman zero,0,0,7,en,False,None
4,shirleyyoung2,NaN,None,None,2011-06-06,543067,1886,957,False,RT @WillBlackWriter: Good @TheTrawlPodcast fro...,0,2,15,en,False,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,healioo,NaN,France,None,2022-02-25,229,3,44,False,RT @Nezzzooo: Andrew Tate Needed to be arreste...,0,6523,13,en,False,"[#LBCPSG, Soler, Sarabia, Gharbi, Châteauroux,..."
96,liltittyhead708,NaN,None,None,2011-02-19,9140,1195,1664,False,RT @Nezzzooo: Andrew Tate Needed to be arreste...,0,6523,13,en,False,None
97,iambeyondfake,NaN,australia 👹,None,2021-05-13,7905,584,574,False,@BreatheNewWinds i thought it would reach 50 p...,0,0,23,en,False,"[#mvcvbri, #MUNEVE, #PAKvNZ, McCarthy, Speaker..."
98,taylor_e_gc,NaN,None,None,2012-04-09,23224,603,1694,False,RT @WickedNFine: Y’all didn’t give a damn abou...,0,107,23,en,True,None


In [6]:
DRIVER.quit()

In [9]:
df.to_csv('test1.csv')

In [11]:
'''
#user data
user_name = [];
location = [];
age = [];
gender = [];
followers = [];
following = [];
verified = []
joined = [] #when they joined
tweet_count_user = []

#tweet data
tweet = [];
word_count = [];
lang = []
likes = []
retweet_count = []
is_quote_status = []
trending_for_area = []

users = [];

tweets_list = [api.get_status(id = 1610297526477025288)]


users = []

DRIVER = webdriver.Chrome()#the chrome

for x in tweets_list:
    
    tweet.append(x.text);
    users.append(x.user);
    word_count.append(countWordsInString(x.text))
    
    lang.append(x.lang)
    likes.append(x.favorite_count) 
    retweet_count.append(x.retweet_count)
    is_quote_status.append(x.is_quote_status)
    
i = 0
for x in users:
    j = x._json
    #opens the page and waits for it to load
    screen_name = j['screen_name']
    country = check_country_real(j['location'])
    
    
    DRIVER.get('https://twitter.com/' + screen_name) 
    
    #get the easy stuff
    user_name.append(screen_name)
    following.append(j['friends_count']);
    followers.append(j['followers_count']);
    location.append(country);
    joined.append(x.created_at.isoformat()[:10])
    tweet_count_user.append(x.statuses_count)
    trending_for_area.append(getTrendingTopics(api,country))
    verified = x.verified
    
    tmp = getAge(DRIVER)#gets the age
    if(tmp == 2023 - x.created_at.year): #checks if the age is the same as the accounts age
        age.append(np.nan)
    else:
        age.append(tmp)
    
    
    gender.append(getGender(j['description'],j['location'],j['name']));
    
    ##this part simply displays at the bottom how many profiles were checked
    clear_output(wait=True) #clears the output
    
    i+=1
    print(i)

print('done')

dick = {"name":user_name,
        "age":age,
        'location':location,
        "gender":gender,
        'join_date':joined,
        'total tweets': tweet_count_user,
        "followers":followers,
        "following":following,
        'verified' : verified,
        'tweet':tweet,
        'likes':likes,
        'retweets':retweet_count,
        'word count': word_count,
        'launguage': lang,
        'is quote':is_quote_status,
        'trending': trending_for_area
       }



df = pd.DataFrame(dick)

df
#'''
print("end")

end
